In [19]:
import pandas as pd
import numpy as np
import requests
import dotenv
import os
import json
dotenv.load_dotenv()

True

In [2]:
data_dir = "football-prediction/epl-training.csv"
df = pd.read_csv(data_dir)

In [3]:
df.columns = [
    'Date',               
    'Home Team',          
    'Away Team',
    'Full Time Home Goals',  
    'Full Time Away Goals',  
    'Full Time Result',       
    'Half Time Home Goals',  
    'Half Time Away Goals',  
    'Half Time Result',      
    'Referee',
    'Home Shots',             
    'Away Shots',             
    'Home Shots on Target',   
    'Away Shots on Target',   
    'Home Corners',          
    'Away Corners',           
    'Home Fouls',             
    'Away Fouls',             
    'Home Yellow Cards',     
    'Away Yellow Cards',     
    'Home Red Cards',     
    'Away Red Cards'      
]

In [20]:
BASE_URL = 'https://api.football-data.org/v4'
HEADERS = {"X-Auth-Token": os.getenv("FOOTBALL_DATA")}

In [32]:
# no need to run again - for getting season information

url = f"{BASE_URL}/competitions/PL"
response = requests.get(url, headers=HEADERS)
data = response.json()  
with open(f"additional_data/pl2.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

In [48]:
# using season information to get match data for that season
with open('additional_data/pl.json', 'r') as file:
    data = json.load(file)

start_dates = []
end_dates = []

for season in data['seasons']:
    start_dates.append(season['startDate'])
    end_dates.append(season['endDate'])

In [49]:
data_end_idx = -1
for i in range(len(start_dates)):
    if start_dates[i] == "2000-08-19":
        data_end_idx = i

start_dates = start_dates[:data_end_idx+1] 
end_dates = end_dates[:data_end_idx+1] 

In [50]:
def get_matches(start_date, end_date):
    url = f"{BASE_URL}/competitions/PL/matches"
    params = {"dateFrom": start_date, "dateTo": end_date}
    response = requests.get(url, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        matches = response.json()["matches"]
        output_dir = "additional_data"
        output_file = os.path.join(output_dir, f"{start_date}_prem_league_matches.json")
        
        with open(output_file, 'w') as file:
            json.dump(matches, file, indent=4)
        print(f"Match data saved to {output_file}")
        return matches
    else:
        print(f"Failed to fetch matches. Status code: {response.status_code}, {response.text}")
        return None

In [44]:
response =requests.get(f"{BASE_URL}/competitions/PL/matches", headers=HEADERS)

In [57]:
for start_date, end_date in list(zip(start_dates, end_dates))[3:]:
    get_matches(start_date, end_date)

Failed to fetch matches. Status code: 403, {"message":"The resource you are looking for is restricted and apparently not within your permissions. Please check your subscription.","errorCode":403}
Failed to fetch matches. Status code: 403, {"message":"The resource you are looking for is restricted and apparently not within your permissions. Please check your subscription.","errorCode":403}
Failed to fetch matches. Status code: 403, {"message":"The resource you are looking for is restricted and apparently not within your permissions. Please check your subscription.","errorCode":403}
Failed to fetch matches. Status code: 403, {"message":"The resource you are looking for is restricted and apparently not within your permissions. Please check your subscription.","errorCode":403}
Failed to fetch matches. Status code: 403, {"message":"The resource you are looking for is restricted and apparently not within your permissions. Please check your subscription.","errorCode":403}
Failed to fetch matc

In [58]:
def get_match_details_for_season(json_file):
    
    match_ids = []
    
    with open(f"additional_data/{json_file}", "r") as file:
        data = json.load(file)

    for match in data:
        match_ids.append(match["id"])
    return match_ids

match_ids = get_match_details_for_season("2022-08-05_prem_league_matches.json")

In [ ]:
def get_match_details(match_id, output_file="match_details.json"):
    url = f"{BASE_URL}/matches/{match_id}"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code == 200:
        match_details = response.json()
        
        # Check if the file exists
        if os.path.exists(output_file):
            with open(output_file, 'r') as file:
                data = json.load(file)
        else:
            data = []
        data.append(match_details)
        
        with open(output_file, 'w') as file:
            json.dump(data, file, indent=4)
        
        print(f"Match details appended to {output_file}")
        return match_details
    else:
        print(f"Failed to fetch match details. Status code: {response.status_code}, {response.text}")
        return None

In [62]:
get_match_details(match_ids[1:])

Failed to fetch match details. Status code: 400
